In [62]:
import pandas as pd
import numpy as np
import datetime
import importlib
import pickle
import time

import data_utils
import model_utils
import train_utils
import evaluation
import submission
importlib.reload(data_utils)
importlib.reload(model_utils)
importlib.reload(train_utils)
importlib.reload(evaluation)

import torch
import torch.nn as nn

### Run inference with Huggingface pretrained bert fine-tuned on SQuAD

In [63]:
file_dict = {"train": "./data/tiny_train.json"}
train_data = data_utils.prep_data(file_dict["train"])

In [66]:
importlib.reload(model_utils)
importlib.reload(train_utils)
m = model_utils.PreTrainedSQuAD().to("cuda")
# _ = train_utils.inference(train_data, m)

In [51]:
answers

{'56cff91b234ae51400d9c1bb': 'nontheistic',
 '56cff91b234ae51400d9c1bc': 'teachings attributed to gautama buddha',
 '56cff91b234ae51400d9c1bd': 'present - day nepal',
 '56cff91b234ae51400d9c1be': 'direct understanding and perception of dependent origination and the four noble truths',
 '56cff91b234ae51400d9c1bf': 'ignorance and craving',
 '56d0c933234ae51400d9c43c': 'nontheistic',
 '56d0c933234ae51400d9c43d': 'gautama buddha',
 '56d0c933234ae51400d9c43e': 'between the 6th and 4th centuries bce',
 '56d0c933234ae51400d9c43f': 'nepal',
 '56d0d6db17492d1400aab662': 'gautama buddha',
 '56d0d6db17492d1400aab663': 'present - day nepal',
 '56d0d6db17492d1400aab664': 'between the 6th and 4th centuries bce',
 '56d1b926e7d4791d009020cd': 'gautama buddha',
 '56d1b926e7d4791d009020cf': 'four',
 '56d1b926e7d4791d009020d0': 'indian',
 '56d1b926e7d4791d009020d1': 'gautama buddha'}

In [52]:
answers.update({1:2})

In [53]:
answers

{'56cff91b234ae51400d9c1bb': 'nontheistic',
 '56cff91b234ae51400d9c1bc': 'teachings attributed to gautama buddha',
 '56cff91b234ae51400d9c1bd': 'present - day nepal',
 '56cff91b234ae51400d9c1be': 'direct understanding and perception of dependent origination and the four noble truths',
 '56cff91b234ae51400d9c1bf': 'ignorance and craving',
 '56d0c933234ae51400d9c43c': 'nontheistic',
 '56d0c933234ae51400d9c43d': 'gautama buddha',
 '56d0c933234ae51400d9c43e': 'between the 6th and 4th centuries bce',
 '56d0c933234ae51400d9c43f': 'nepal',
 '56d0d6db17492d1400aab662': 'gautama buddha',
 '56d0d6db17492d1400aab663': 'present - day nepal',
 '56d0d6db17492d1400aab664': 'between the 6th and 4th centuries bce',
 '56d1b926e7d4791d009020cd': 'gautama buddha',
 '56d1b926e7d4791d009020cf': 'four',
 '56d1b926e7d4791d009020d0': 'indian',
 '56d1b926e7d4791d009020d1': 'gautama buddha',
 1: 2}

In [46]:
i.id

['56cff91b234ae51400d9c1bb',
 '56cff91b234ae51400d9c1bc',
 '56cff91b234ae51400d9c1bd',
 '56cff91b234ae51400d9c1be',
 '56cff91b234ae51400d9c1bf',
 '56d0c933234ae51400d9c43c',
 '56d0c933234ae51400d9c43d',
 '56d0c933234ae51400d9c43e',
 '56d0c933234ae51400d9c43f',
 '56d0d6db17492d1400aab662',
 '56d0d6db17492d1400aab663',
 '56d0d6db17492d1400aab664',
 '56d1b926e7d4791d009020cd',
 '56d1b926e7d4791d009020cf',
 '56d1b926e7d4791d009020d0',
 '56d1b926e7d4791d009020d1']

In [32]:
i.a

['nontheistic',
 'teachings attributed to Gautama Buddha',
 'present-day Nepal',
 'through the direct understanding and perception of dependent origination and the Four Noble Truths',
 'ignorance and craving',
 'nontheistic religion',
 'Gautama Buddha',
 'between the 6th and 4th centuries BCE',
 'present-day Nepal',
 'Gautama Buddha',
 'Nepal',
 'sometime between the 6th and 4th centuries BCE',
 'Gautama Buddha',
 '4',
 'Indian',
 'Buddha']

In [33]:
answers

['nontheistic',
 'teachings attributed to gautama buddha',
 'present - day nepal',
 'direct understanding and perception of dependent origination and the four noble truths',
 'ignorance and craving',
 'nontheistic',
 'gautama buddha',
 'between the 6th and 4th centuries bce',
 'nepal',
 'gautama buddha',
 'present - day nepal',
 'between the 6th and 4th centuries bce',
 'gautama buddha',
 'four',
 'indian',
 'gautama buddha']